In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install imblearn


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the data
file_path = Path('Data/Supervised/pct_cr_etf_5_years_complete_trail_df.csv')
df = pd.read_csv(file_path)

# Preview the data
df.set_index('ETF_Names')
df.head()

,ETF_Names,30-Jun-19,31-Jul-19,31-Aug-19,30-Sep-19,31-Oct-19,30-Nov-19,31-Dec-19,31-Jan-20,29-Feb-20,...,31-Mar-22,30-Apr-22,31-May-22,30-Jun-22,STD_2019_2020,STD_2020_2021,STD_2021_2022,Class,PCT_Cumulative_Returns,Target_met
0,iShares Asia 50 ETF,6.81,-0.84,-2.85,3.24,2.12,2.52,4.41,-0.92,3.41,...,-8.37,-0.80,0.93,0.42,3.518415,3.015198,4.323178,0,-5.930000e+13,False
1,iShares China Large-Cap ETF,5.44,-2.18,-2.63,1.63,1.22,1.34,4.14,-4.36,6.23,...,-11.48,2.27,2.01,11.45,3.488395,3.665791,7.037962,0,-3.240000e+15,False
2,iShares Core Cash ETF,0.13,0.13,0.08,0.09,0.08,0.08,0.08,0.09,0.07,...,0.00,-0.01,0.03,0.05,0.036829,0.005189,0.016132,2,0.000000e+00,False
3,iShares Core Composite Bond ETF,1.01,0.96,1.49,-0.49,-0.51,0.80,-1.65,2.31,0.84,...,-3.78,-1.52,-0.90,-1.49,1.014293,1.168062,1.755788,2,3.740000e-05,False
4,iShares Core Global Corporate Bond (AUD Hedged...,2.31,0.88,2.01,-0.51,0.20,0.16,0.24,1.98,0.04,...,-1.50,-4.65,-0.18,-2.98,2.547621,1.253187,1.724990,5,6.290000e-05,False


In [4]:
# Create our features
X = pd.get_dummies(df.drop(["ETF_Names", 
                            "STD_2019_2020", 
                            "STD_2020_2021", 
                            "STD_2021_2022", 
                            "Class", 
                            "PCT_Cumulative_Returns"],
                             axis=1))

# Create our target
y = df[["PCT_Cumulative_Returns"]]

In [5]:
X.describe()

,30-Jun-19,31-Jul-19,31-Aug-19,30-Sep-19,31-Oct-19,30-Nov-19,31-Dec-19,31-Jan-20,29-Feb-20,31-Mar-20,...,30-Sep-21,31-Oct-21,30-Nov-21,31-Dec-21,31-Jan-22,28-Feb-22,31-Mar-22,30-Apr-22,31-May-22,30-Jun-22
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,...,32.000000,32.000000,32.000000,32.00000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,3.879687,1.266563,-0.541250,1.704063,0.315000,2.554375,0.011250,2.041875,-3.826250,-9.714062,...,-2.648750,-0.039375,1.485000,1.75875,-2.494687,-2.814062,-0.405938,-1.902188,-0.970000,-4.211563
std,2.072361,1.974436,2.130842,1.628405,1.262356,1.791962,2.200167,2.542379,4.046414,7.321310,...,1.902233,2.964479,2.136514,2.25297,3.406237,3.339546,4.613504,3.168000,1.972797,4.554291
min,-0.210000,-5.790000,-3.830000,-0.960000,-2.770000,-0.680000,-3.110000,-4.360000,-9.800000,-22.470000,...,-5.860000,-6.360000,-2.350000,-5.64000,-9.010000,-10.680000,-11.480000,-9.450000,-7.060000,-13.230000
25%,2.587500,0.390000,-2.367500,0.220000,-0.525000,1.270000,-1.670000,0.097500,-6.467500,-14.952500,...,-3.860000,-1.217500,-0.307500,0.06000,-4.387500,-4.900000,-3.555000,-3.147500,-1.822500,-7.237500
50%,4.140000,1.445000,-0.400000,1.785000,0.115000,2.545000,-0.685000,2.300000,-4.840000,-9.740000,...,-2.710000,-0.195000,1.390000,2.03500,-2.625000,-2.765000,-0.270000,-1.485000,-0.200000,-4.465000
75%,5.472500,2.327500,0.722500,2.972500,1.225000,3.417500,1.640000,3.812500,-0.082500,-4.427500,...,-1.857500,0.465000,2.885000,3.20750,-0.912500,-0.907500,3.310000,-0.017500,0.130000,-1.450000
max,6.810000,4.410000,4.370000,5.670000,2.750000,6.410000,4.410000,5.870000,6.230000,1.830000,...,3.910000,6.910000,5.310000,5.80000,7.020000,2.650000,8.010000,6.330000,2.010000,11.450000


In [6]:
# Check the balance of our target values
y['PCT_Cumulative_Returns'].value_counts()

 0.000000e+00    2
-5.930000e+13    1
 1.729352e-01    1
-1.670000e+13    1
 9.020000e+14    1
-8.940000e+13    1
 7.120000e+17    1
 9.270000e+14    1
 4.080000e+13    1
-8.230000e+19    1
 2.490000e+18    1
-7.120000e+14    1
-4.443492e+09    1
 3.597110e+09    1
-4.519803e+08    1
-4.966301e+03    1
-4.510000e+11    1
-3.240000e+15    1
 1.720000e+13    1
 1.405238e+10    1
 5.290000e+17    1
-3.464153e+10    1
-5.670000e+12    1
-4.410000e+14    1
 3.390000e+15    1
-3.790000e+15    1
 2.590000e+13    1
 2.230000e+20    1
 6.290000e-05    1
 3.740000e-05    1
 2.316679e-03    1
Name: PCT_Cumulative_Returns, dtype: int64

In [7]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [10]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [11]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [14]:
# Resample the training data with the BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'

In [ ]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

In [ ]:
# Train the Classifier
eec = EasyEnsembleClassifier(n_estimators=100,random_state=1)
eec.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))